In [35]:
import torchaudio
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration

model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B", torch_dtype="auto", device_map="auto")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B", trust_remote_code=True)

2025-05-23 15:29:06,409 WARNING The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
2025-05-23 15:29:06,411 INFO Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 1: 404791296 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

2025-05-23 15:29:07,608 WARNING Some parameters are on the meta device because they were offloaded to the cpu.


In [39]:
prompt = "<|audio_bos|><|AUDIO|><|audio_eos|>Describe the music fragment in detail:"
audio, sr = torchaudio.load("/home/mszawerda/music-sae/data/input/music-bench/datashare/data/_KYo_89lgf0.wav")
transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=processor.feature_extractor.sampling_rate)
audio = transform(audio)
inputs = processor(text=[prompt] * 3, audios=[audio[0].numpy()] * 3, return_tensors="pt")

generated_ids = model.generate(**{k: v.to("cuda") for k, v in inputs.items()}, max_length=300)
generated_ids = generated_ids[:, inputs.input_ids.size(1) :]
response = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
response

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


[' The low quality recording features a male vocal singing over a very loud crowd clapping and cheering, while some of them are making scary sounds. It sounds like a live performance and it is very energetic.',
 ' The low quality recording features a male vocal singing over a very loud crowd clapping and cheering, while some of them are making scary sounds. It sounds like a live performance and it is very energetic.',
 ' The low quality recording features a male vocal singing over a very loud crowd clapping and cheering, while some of them are making scary sounds. It sounds like a live performance and it is very energetic.']